# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = 'output_data/cities.csv'
map_df = pd.read_csv(path)
map_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Oktyabr'skiy,54.4815,53.4710,50.07,64,100,15.77,RU,2021-09-24 05:29:17
1,1,Bubaque,11.2833,-15.8333,80.87,77,100,4.14,GW,2021-09-24 05:29:17
2,2,Avarua,-21.2078,-159.7750,71.65,68,75,4.61,CK,2021-09-24 05:29:18
3,3,Hermanus,-34.4187,19.2345,55.90,87,46,7.27,ZA,2021-09-24 05:29:18
4,4,Santa Fe,-31.6333,-60.7000,58.01,86,0,1.99,AR,2021-09-24 05:29:13
...,...,...,...,...,...,...,...,...,...,...
541,541,Atotonilco el Grande,20.2833,-98.6667,53.33,94,90,2.17,MX,2021-09-24 05:33:00
542,542,Sangar,63.9242,127.4739,42.87,93,100,4.43,RU,2021-09-24 05:33:00
543,543,Mastic Beach,40.7668,-72.8521,75.38,88,90,3.00,US,2021-09-24 05:33:01
544,544,Mossamedes,-15.1961,12.1522,67.17,92,39,0.96,AO,2021-09-24 05:33:01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Gmaps configuration 
gmaps.configure(api_key=g_key)
# Lat and Lng stored as locations
locations = map_df[["Latitude", "Longitude"]].astype(float)
# Humidity stored as weight
weight = map_df["Humidity"].astype(float)
# Map Style
with open('resources/map.json') as map:
    geometry = json.load(map)
# Create google map figure
fig = gmaps.figure(zoom_level=2, center=(40,-100), layout={'width': '1050px', 'height': '700px', 'margin': '0 auto 0 auto'})
# # GeoJSON
# geojson_layer = gmaps.geojson_layer(geometry)
# Add heat map layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100, point_radius=2)
# Initialize Map
fig.add_layer(heat_layer)
# Show
fig

Figure(layout=FigureLayout(height='700px', margin='0 auto 0 auto', width='1050px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Temp 60-80 F
# Cloudiness > 30%
# Wind Speed < 10mph
# Humidity < 20%
map_df = hotel_df = map_df[(map_df['Max Temperature'] > 60) & (map_df['Max Temperature'] < 80) & (map_df['Wind Speed'] <= 10) & (map_df['Cloudiness'] >= 30) & (map_df['Humidity'] < 60)]
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,Kumul,42.8000,93.4500,78.06,13,36,3.44,CN,2021-09-24 05:29:29
66,66,Hegang,47.4000,130.3667,63.27,40,53,5.99,CN,2021-09-24 05:29:43
73,73,Victor Harbor,-35.5667,138.6167,61.92,51,66,4.00,AU,2021-09-24 05:29:46
93,93,Airai,-8.9266,125.4092,78.24,37,91,2.53,TL,2021-09-24 05:29:54
100,100,Novo Oriente,-5.5344,-40.7742,78.21,52,75,6.73,BR,2021-09-24 05:29:56
167,167,Liuhe,42.2401,125.7172,71.51,56,85,1.54,CN,2021-09-24 05:30:26
190,190,Faya,18.3851,42.4509,70.32,24,39,5.82,SA,2021-09-24 05:30:36
268,268,Luena,-11.7833,19.9167,70.41,23,98,3.09,AO,2021-09-24 05:31:08
287,287,Zastron,-30.3023,27.0840,62.17,31,97,6.55,ZA,2021-09-24 05:31:16
300,300,Grants,35.1473,-107.8514,64.63,32,75,6.91,US,2021-09-24 05:31:21


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Build Dict in order to use loc
dict_city = hotel_df['City'].tolist()
dict_lat = hotel_df['Latitude'].tolist()
dict_lon = hotel_df['Longitude'].tolist()
dict_country = hotel_df['Country'].tolist()
dict_date = hotel_df['Date'].tolist()
# Create Dict
dict = {'City': dict_city, 'Latitude': dict_lat, 'Longitude': dict_lon, 'Country': dict_country, 'Date': dict_date}
# Covert dict to DataFrame
hotel_df = pd.DataFrame.from_dict(dict)

In [6]:
# Add url for Google Places API
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Set params
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}
# Loop through 'hotel_df'
for index, row in hotel_df.iterrows():
    # Location params
    lat = row['Latitude']
    lon = row['Longitude']
    # Updates location params whilst looping
    params['location'] = f'{lat},{lon}'
    # Make api call
    response = requests.get(url, params=params).json()
    # Try to append the hotels requested to appropriate columns
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except(KeyError, IndexError):
        print('Missing Hotel Info..Skipping...')
hotel_df    

Missing Hotel Info..Skipping...
Missing Hotel Info..Skipping...
Missing Hotel Info..Skipping...


,City,Latitude,Longitude,Country,Date,Hotel Name
0,Kumul,42.8000,93.4500,CN,2021-09-24 05:29:29,Yushang Hotel
1,Hegang,47.4000,130.3667,CN,2021-09-24 05:29:43,NaN
2,Victor Harbor,-35.5667,138.6167,AU,2021-09-24 05:29:46,Grosvenor Hotel
3,Airai,-8.9266,125.4092,TL,2021-09-24 05:29:54,NaN
4,Novo Oriente,-5.5344,-40.7742,BR,2021-09-24 05:29:56,Hotel brasil
5,Liuhe,42.2401,125.7172,CN,2021-09-24 05:30:26,Northern Hotel
6,Faya,18.3851,42.4509,SA,2021-09-24 05:30:36,NaN
7,Luena,-11.7833,19.9167,AO,2021-09-24 05:31:08,Hospedagem Dilvinha
8,Zastron,-30.3023,27.0840,ZA,2021-09-24 05:31:16,36 on Boom B&B
9,Grants,35.1473,-107.8514,US,2021-09-24 05:31:21,Quality Inn & Suites


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations=locations, info_box_content=hotel_info)
# Add markers
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='700px', margin='0 auto 0 auto', width='1050px'))